<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/notebooks/SpeechSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# F5-TTS
!pip install -q f5-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

# Data

In [ ]:
# Reference Speech
!gdown 1uTreohCIiYSlrQTa3fuH1_IQjdeW1SaE --quiet
# Reference Meta (including Text)
!gdown 1PBs6r3cqhFxWzy9s5wrGpF6-5ZnoOWxE --quiet

!unzip -q audio.zip -d audio
!unzip -q json.zip -d json

In [ ]:
import gdown

gid = '1cgTo9px9Wd0fYcbjvTkc1W3r-t5Pb4aI'
url = f"https://drive.google.com/uc?id={gid}"

output = gdown.download(url, quiet=False)  # output sẽ là tên file đã tải


Downloading...
From: https://drive.google.com/uc?id=1cgTo9px9Wd0fYcbjvTkc1W3r-t5Pb4aI
To: /content/sample-500-text-2.csv
100%|██████████| 34.8k/34.8k [00:00<00:00, 48.7MB/s]


# Model

In [ ]:
import pandas as pd
text_command_df = pd.read_csv(output)

# Pipeline

In [ ]:
def audio_generate(command: str, model = f5tts, ref_file: str = "", ref_text: str = ""):
  '''
  generate speech command with model f5tts
  '''
  wav, sr, spec = model.infer(
      ref_file=ref_file,
      ref_text=ref_text,
      gen_text=command,
      seed=None,
  )
  return wav, sr, spec

def ref_sample(audio_folder_path, json_folder_path):
  '''
  sampling reference (audio and text)

  Returns:
  - id: id of reference (for tracing)
  - ref_speech_path: path of reference audio
  - ref_text_path: path of reference text

  '''

  # list for sampling
  audio_files = os.listdir(audio_folder_path)
  json_files = os.listdir(json_folder_path)

  # sampling
  audio_file = random.choice(audio_files)
  json_file = random.choice(json_files)

  # get ID
  id = json_file.split(".")[0]

  # get ref path
  ref_speech_path = os.path.join(audio_folder_path, audio_file)
  ref_text_path = os.path.join(json_folder_path, json_file)

  return id, ref_speech_path, ref_text_path

In [ ]:
def audio_generate(command: str, model = f5tts, ref_file: str = "", ref_text: str = ""):
  '''
  generate speech command with model f5tts
  '''
  wav, sr, spec = model.infer(
      ref_file=ref_file,
      ref_text=ref_text,
      gen_text=command,
      seed=None,
  )
  return wav, sr, spec

def ref_sample(audio_folder_path, json_folder_path):
  '''
  sampling reference (audio and text)

  Returns:
  - id: id of reference (for tracing)
  - ref_speech_path: path of reference audio
  - ref_text_path: path of reference text

  '''

  # list for sampling
  audio_files = os.listdir(audio_folder_path)
  json_files = os.listdir(json_folder_path)

  # sampling
  audio_file = random.choice(audio_files)
  json_file = random.choice(json_files)

  # get ID
  id = json_file.split(".")[0]

  # get ref path
  ref_speech_path = os.path.join(audio_folder_path, audio_file)
  ref_text_path = os.path.join(json_folder_path, json_file)

  return id, ref_speech_path, ref_text_path

import os
import re

import os
import json
import soundfile as sf
import pandas as pd
import random
import ast
import numpy as np
import uuid

def command_synthesis_pipeline(
    text_command: pd.DataFrame,
    ref_folder_path: str,
    json_folder_path: str,
    export_path: str = 'synthesis_command'
) -> pd.DataFrame:
    """
    Generate synthetic audio commands based on provided text commands and reference samples.

    This function processes a set of text commands, synthesizes corresponding audio files
    using reference speech and text samples, and stores both audio and metadata in a structured folder format.

    Parameters
    ----------
    text_command : pd.DataFrame
        DataFrame containing command data. Must include:
        - 'command' (str): The text of the command.
        - 'type' (str): The command type/category.
        - 'segments' (list or None): Optional list of segment identifiers.
    ref_folder_path : str
        Path to the folder containing reference audio files.
    json_folder_path : str
        Path to the folder containing JSON files with reference text.
    export_path : str, optional
        Output folder for synthesized commands (default: 'synthesis_command').

    Returns
    -------
    pd.DataFrame
        A DataFrame summarizing all synthesized commands and metadata.

    Notes
    -----
    - Requires external helper functions:
      `get_command_count(export_path)`, `ref_sample(ref_folder_path, json_folder_path)`,
      and `audio_generate(command, ref_file, ref_text)`.
    - Each generated command will be stored in a subfolder named
      `{type}_{sequence_number}` containing:
        - Full synthesized WAV file
        - Optional segmented WAV files
        - Corresponding JSON metadata
    """
    os.makedirs(export_path, exist_ok = True)
    # Track current command count per type
    #### cur_command_count = get_command_count(export_path)
    list_synthesis_command = []

    for _, row in text_command.iterrows():
        command = row['text']

        # Select reference sample
        ref_id, ref_speech_path, ref_text_path = ref_sample(ref_folder_path, json_folder_path)

        # Generate full audio
        wav, sr, spec = audio_generate(command=command, ref_file=ref_speech_path, ref_text=ref_text_path)

        # Ensure export directory exists
        os.makedirs(export_path, exist_ok=True)

        # Increment command counter for this type

        #### cur_command_count[row['type']] += 1
        cur_command_count_str = uuid.uuid4().hex[:8]

        # Create folder for this command
        command_path = os.path.join(export_path, f"{row['type']}_{cur_command_count_str}")
        os.makedirs(command_path, exist_ok=True)

        # Save full audio
        sf.write(
            os.path.join(command_path, f"{row['type']}_{cur_command_count_str}_full.wav"),
            wav,
            sr
        )

        # Save segmented audio if segments exist
        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:
            seg_list = ast.literal_eval(row['segments'])
            for count, seg in enumerate(seg_list):
                seg_wav, seg_sr, _ = audio_generate(command=seg[str(count)], ref_file=ref_speech_path, ref_text=ref_text_path)
                sf.write(
                    os.path.join(command_path, f"{row['type']}_{cur_command_count_str}_seg_{count}.wav"),
                    seg_wav,
                    seg_sr
                )

        type_segments = {}
        text_segments = {}
        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:

            for seg_item in ast.literal_eval(row['segments']):
                list_key = list(seg_item.keys())
                type_segments[list_key[0]] = seg_item[list_key[1]]
                text_segments[list_key[0]] = seg_item[list_key[0]]
        # Create metadata JSON
        json_template = {
            "id": cur_command_count_str,
            "type": row['type'],
            "command": command,
            "sampling_rate": sr,
            "num_segments": len(type_segments),
            "type_segments": type_segments,
            "text_segments": text_segments,
            "ref_id": ref_id,
            "ref_file": ref_speech_path,
            "ref_text": ref_text_path
        }

        with open(os.path.join(command_path, f"{row['type']}_{cur_command_count_str}.json"), 'w') as f:
            json.dump(json_template, f, ensure_ascii=False, indent=2)

        list_synthesis_command.append(json_template)

    # Return summary DataFrame
    return pd.DataFrame(list_synthesis_command)


In [ ]:
import os
import json
import soundfile as sf
import pandas as pd
import random
import ast
import numpy as np
import uuid

def command_synthesis_pipeline(
    text_command: pd.DataFrame,
    ref_folder_path: str,
    json_folder_path: str,
    export_path: str = 'synthesis_command'
) -> pd.DataFrame:
    """
    Generate synthetic audio commands based on provided text commands and reference samples.

    This function processes a set of text commands, synthesizes corresponding audio files
    using reference speech and text samples, and stores both audio and metadata in a structured folder format.

    Parameters
    ----------
    text_command : pd.DataFrame
        DataFrame containing command data. Must include:
        - 'command' (str): The text of the command.
        - 'type' (str): The command type/category.
        - 'segments' (list or None): Optional list of segment identifiers.
    ref_folder_path : str
        Path to the folder containing reference audio files.
    json_folder_path : str
        Path to the folder containing JSON files with reference text.
    export_path : str, optional
        Output folder for synthesized commands (default: 'synthesis_command').

    Returns
    -------
    pd.DataFrame
        A DataFrame summarizing all synthesized commands and metadata.

    Notes
    -----
    - Requires external helper functions:
      `get_command_count(export_path)`, `ref_sample(ref_folder_path, json_folder_path)`,
      and `audio_generate(command, ref_file, ref_text)`.
    - Each generated command will be stored in a subfolder named
      `{type}_{sequence_number}` containing:
        - Full synthesized WAV file
        - Optional segmented WAV files
        - Corresponding JSON metadata
    """
    os.makedirs(export_path, exist_ok = True)
    # Track current command count per type
    #### cur_command_count = get_command_count(export_path)
    list_synthesis_command = []

    for _, row in text_command.iterrows():
        command = row['text']

        # Select reference sample
        ref_id, ref_speech_path, ref_text_path = ref_sample(ref_folder_path, json_folder_path)

        # Generate full audio
        wav, sr, spec = audio_generate(command=command, ref_file=ref_speech_path, ref_text=ref_text_path)

        # Ensure export directory exists
        os.makedirs(export_path, exist_ok=True)

        # Increment command counter for this type

        #### cur_command_count[row['type']] += 1
        cur_command_count_str = uuid.uuid4().hex[:8]

        # Create folder for this command
        command_path = os.path.join(export_path, f"{row['type']}_{cur_command_count_str}")
        os.makedirs(command_path, exist_ok=True)

        # Save full audio
        sf.write(
            os.path.join(command_path, f"{row['type']}_{cur_command_count_str}_full.wav"),
            wav,
            sr
        )

        # Save segmented audio if segments exist
        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:
            seg_list = ast.literal_eval(row['segments'])
            for count, seg in enumerate(seg_list):
                seg_wav, seg_sr, _ = audio_generate(command=seg[str(count)], ref_file=ref_speech_path, ref_text=ref_text_path)
                sf.write(
                    os.path.join(command_path, f"{row['type']}_{cur_command_count_str}_seg_{count}.wav"),
                    seg_wav,
                    seg_sr
                )

        type_segments = {}
        text_segments = {}
        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:

            for seg_item in ast.literal_eval(row['segments']):
                list_key = list(seg_item.keys())
                type_segments[list_key[0]] = seg_item[list_key[1]]
                text_segments[list_key[0]] = seg_item[list_key[0]]
        # Create metadata JSON
        json_template = {
            "id": cur_command_count_str,
            "type": row['type'],
            "command": command,
            "sampling_rate": sr,
            "num_segments": len(type_segments),
            "type_segments": type_segments,
            "text_segments": text_segments,
            "ref_id": ref_id,
            "ref_file": ref_speech_path,
            "ref_text": ref_text_path
        }

        with open(os.path.join(command_path, f"{row['type']}_{cur_command_count_str}.json"), 'w') as f:
            json.dump(json_template, f, ensure_ascii=False, indent=2)

        list_synthesis_command.append(json_template)

    # Return summary DataFrame
    return pd.DataFrame(list_synthesis_command)


In [ ]:
df_commands_synthesis = command_synthesis_pipeline(
    text_command=text_command_df,
    ref_folder_path='audio/dung',
    json_folder_path='json/dung',
    export_path='synthesis_command'
)

Converting audio...
Using custom reference text...

ref_text   json/dung/EN_B00013_S02252_W000000.json. 
gen_text 0 I heard that Tubi has a lot of great shows right now.


Generating audio in 1 batches...


100%|██████████| 1/1 [00:10<00:00, 10.74s/it]


Converting audio...
Using custom reference text...

ref_text   json/dung/EN_B00013_S02742_W000000.json. 
gen_text 0 switch to channel ABC


Generating audio in 1 batches...


  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

# Optimize Code

In [ ]:
import os
import json
import soundfile as sf
import pandas as pd
import random
import ast
import numpy as np
import uuid
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
from pathlib import Path
import time
from typing import Any, List

# ==== Giả sử bạn có hàm load_model riêng ====
def load_model():
    from importlib.resources import files
    from f5_tts.api import F5TTS

    f5tts = F5TTS(device = 'cuda')
    return f5tts


def audio_generate(command: str, model, ref_file: str = "", ref_text: str = ""):
    wav, sr, spec = model.infer(
        ref_file=ref_file,
        ref_text=ref_text,
        gen_text=command,
        seed=None,
    )
    return wav, sr, spec


class ReferenceCache:
    def __init__(self, audio_folder_path: str, json_folder_path: str):
        self.audio_folder_path = Path(audio_folder_path)
        self.json_folder_path = Path(json_folder_path)
        self._audio_files = None
        self._json_files = None
        self._ref_texts = {}

    @property
    def audio_files(self):
        if self._audio_files is None:
            self._audio_files = [f for f in self.audio_folder_path.iterdir() if f.is_file()]
        return self._audio_files

    @property
    def json_files(self):
        if self._json_files is None:
            self._json_files = [f for f in self.json_folder_path.iterdir() if f.suffix == '.json']
        return self._json_files

    def get_ref_text(self, json_path: Path) -> str:
        if str(json_path) not in self._ref_texts:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self._ref_texts[str(json_path)] = data.get('text', '')
        return self._ref_texts[str(json_path)]

    def sample_reference(self):
        audio_file = random.choice(self.audio_files)
        json_file = random.choice(self.json_files)
        ref_id = json_file.stem
        ref_speech_path = str(audio_file)
        ref_text = self.get_ref_text(json_file)
        return ref_id, ref_speech_path, ref_text


def process_single_command(row: pd.Series, ref_cache: ReferenceCache, export_path: str, model: Any):
    try:
        ref_id, ref_speech_path, ref_text = ref_cache.sample_reference()
        wav, sr, _ = audio_generate(row['text'], model, ref_speech_path, ref_text)

        cur_id = uuid.uuid4().hex[:8]
        cmd_path = Path(export_path) / f"{row['type']}_{cur_id}"
        cmd_path.mkdir(parents=True, exist_ok=True)

        # Save full audio
        sf.write(cmd_path / f"{row['type']}_{cur_id}_full.wav", wav, sr)

        type_segments = {}
        text_segments = {}

        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:
            seg_list = ast.literal_eval(row['segments'])
            for count, seg in enumerate(seg_list):
                key = list(seg.keys())[0]
                seg_wav, seg_sr, _ = audio_generate(seg[key], model, ref_speech_path, ref_text)
                sf.write(cmd_path / f"{row['type']}_{cur_id}_seg_{count}.wav", seg_wav, seg_sr)
                list_key = list(seg.keys())
                type_segments[list_key[0]] = seg[list_key[1]] if len(list_key) > 1 else ""
                text_segments[list_key[0]] = seg[list_key[0]]

        json_template = {
            "id": cur_id,
            "type": row['type'],
            "command": row['text'],
            "sampling_rate": sr,
            "num_segments": len(type_segments),
            "type_segments": type_segments,
            "text_segments": text_segments,
            "ref_id": ref_id,
            "ref_file": ref_speech_path,
            "ref_text": ref_text
        }
        with open(cmd_path / f"{row['type']}_{cur_id}.json", 'w', encoding='utf-8') as f:
            json.dump(json_template, f, ensure_ascii=False, indent=2)

        return json_template
    except Exception as e:
        print(f"Failed to process: {row['text']}, error: {e}")
        return None


def worker_task(rows: List[pd.Series], ref_folder: str, json_folder: str, export_path: str):
    # Mỗi worker load model riêng
    model = load_model()
    ref_cache = ReferenceCache(ref_folder, json_folder)
    results = []
    for row in rows:
        res = process_single_command(row, ref_cache, export_path, model)
        if res:
            results.append(res)
    return results


def command_synthesis_pipeline_multi_model(
    text_command: pd.DataFrame,
    ref_folder_path: str,
    json_folder_path: str,
    export_path: str = 'synthesis_command',
    num_workers: int = None
):
    start_time = time.time()
    os.makedirs(export_path, exist_ok=True)

    if num_workers is None:
        num_workers = min(mp.cpu_count(), 4)  # Giới hạn số model load

    # Chia dữ liệu thành N phần, mỗi worker xử lý riêng
    chunks = np.array_split(text_command, num_workers)

    all_results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [
            executor.submit(worker_task, chunk.to_dict('records'), ref_folder_path, json_folder_path, export_path)
            for chunk in chunks
        ]
        for i, future in enumerate(futures):
            res_list = future.result()
            all_results.extend(res_list)
            print(f"Worker {i+1} done ({len(res_list)} items)")

    print(f"Pipeline finished in {time.time() - start_time:.2f}s")
    return pd.DataFrame(all_results)


In [ ]:
df_commands_synthesis = command_synthesis_pipeline_multi_model(
    text_command=text_command_df,
    ref_folder_path="audio/dung",
    json_folder_path="json/dung",
    export_path="synthesis_command",
    num_workers=4,        # Số luồng xử lý
)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



vocab :  /usr/local/lib/python3.11/dist-packages/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_v1_Base/model_1250000.safetensors 


vocab :  /usr/local/lib/python3.11/dist-packages/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_v1_Base/model_1250000.safetensors 


vocab :  /usr/local/lib/python3.11/dist-packages/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_v1_Base/model_1250000.safetensors 


vocab :  /usr/local/lib/python3.11/dist-packages/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_v1_Base/model_1250000.safete

  0%|          | 0/1 [00:00<?, ?it/s]

...

Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    We got hired to do a wedding and one of the guests was a Netflix executive. She mentioned they had a new show coming out and asked us if we were interested in auditioning. 
gen_text 0 I really enjoyed our dinner last night?


Generating audio in 1 batches...


100%|██████████| 1/1 [00:03<00:00,  3.81s/it]


Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    No idea where she is? Nope. I haven't seen her since she was here last night. She may be gone for good. But she can't be. All her stuff is there. Clothes and everything. 
gen_text 0 turn on subtitles, resume, and decrease the volume


Generating audio in 1 batches...


100%|██████████| 1/1 [00:02<00:00,  2.20s/it]

Worker 1 done (51 items)
Worker 2 done (51 items)
Worker 3 done (51 items)
Worker 4 done (50 items)
Pipeline finished in 1506.98s


In [ ]:
df_commands_synthesis.to_csv('sample-500-speech-2.csv')

In [ ]:
!zip synthesis_command.zip -r synthesis_command

  adding: synthesis_command/ (stored 0%)
  adding: synthesis_command/single_mix_97a2f003/ (stored 0%)
  adding: synthesis_command/single_mix_97a2f003/single_mix_97a2f003_full.wav (deflated 17%)
  adding: synthesis_command/single_mix_97a2f003/single_mix_97a2f003_seg_1.wav (deflated 22%)
  adding: synthesis_command/single_mix_97a2f003/single_mix_97a2f003_seg_0.wav (deflated 55%)
  adding: synthesis_command/single_mix_97a2f003/single_mix_97a2f003.json (deflated 48%)
  adding: synthesis_command/single_mix_f15526ee/ (stored 0%)
  adding: synthesis_command/single_mix_f15526ee/single_mix_f15526ee_seg_1.wav (deflated 92%)
  adding: synthesis_command/single_mix_f15526ee/single_mix_f15526ee_full.wav (deflated 41%)
  adding: synthesis_command/single_mix_f15526ee/single_mix_f15526ee_seg_0.wav (deflated 50%)
  adding: synthesis_command/single_mix_f15526ee/single_mix_f15526ee.json (deflated 48%)
  adding: synthesis_command/single_mix_392d1b1e/ (stored 0%)
  adding: synthesis_command/single_mix_392d1